In [2]:
import pysam
from Bio import SeqIO
import pandas as pd

In [3]:
#alignement = pysam.AlignmentFile("../outputs/zymo/metaMDBG/reads_on_contigs.bam", "rb")
#contigs = pysam.FastaFile("../outputs/zymo/metaMDBG/assembly.fasta")
#reads = pysam.FastaFile("/groups/genscale/nimauric/long_reads/zymoD6331.fastq")

alignement_path = "../outputs/salad-irg/metaMDBG/reads_on_contigs.bam"
contigs_path = "../outputs/salad-irg/metaMDBG/assembly.fasta"
reads_path = "/groups/genscale/nimauric/long_reads/SMRTcell1-M8-fev-sal-irg-1.hifi_reads.fastq"
threshold = 500


In [15]:
### Calculate the range of covered positions
covered_positions = {}
with pysam.AlignmentFile(alignement_path, "rb") as bam:
    for alignment in bam.fetch() :
        read_name = alignment.query_name
        start_pos = alignment.reference_start
        end_pos = alignment.reference_end

        if read_name in covered_positions :
            # In case of multiple alignements, trim the new alignement
            for pos_range in covered_positions[read_name]:
                # Trim the begining
                if start_pos >= pos_range[0] and start_pos < pos_range[1] :
                    start_pos = pos_range[1]

                # Trim the end
                if end_pos > pos_range[0] and end_pos <= pos_range[1] :
                    end_pos = pos_range[0]

            # If not empty, add the new alignement
            if start_pos > end_pos : 
                covered_positions[read_name].append((start_pos, end_pos))

        else:
            covered_positions[read_name] = [(start_pos, end_pos)]



[W::hts_idx_load3] The index file is older than the data file: ../outputs/salad-irg/metaMDBG/reads_on_contigs.bam.bai


In [26]:
# Calculate the number of aligned reads and the total alignement length from those positions
alignements_length = 0
aligned_reads_count = 0

for p in covered_positions.values() : 
    tmp_length = 0
    for a in p : 
        tmp_length += a[1] - a[0]
    
    alignements_length += tmp_length
    if(tmp_length > threshold) : 
        aligned_reads_count +=1

print(aligned_reads_count, alignements_length)





1983671 7460061174


In [17]:
# Calculate the total number of reads and the total read length
reads_length = 0
reads_count = 0

with open(reads_path, "r") as fastq :
    for record in SeqIO.parse(fastq, "fastq") :
        reads_length += len(record.seq)
        reads_count += 1
print(reads_count, reads_length)

2913016 20251105762


In [27]:
# Calculate the ratio

print("aligned reads over total reads count ratio : {:.2f}% ({}/{})".format(aligned_reads_count / reads_count * 100,aligned_reads_count, reads_count ))
print("alignement length over total reads length ratio : {:.2f}% ({}/{})".format(alignements_length / reads_length * 100, alignements_length, reads_length))

aligned reads over total reads count ratio : 68.10% (1983671/2913016)
alignement length over total reads length ratio : 36.84% (7460061174/20251105762)
